In [74]:
# lets import the required libraries

# for mathemaical operations
import numpy as np
# for dataframe manipulations
import pandas as pd
import seaborn as sb
import streamlit as st


# for data visualizations
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pickle

from imblearn.over_sampling import RandomOverSampler
#from scipy.signal._signaltools import _centeredd
# Let's create a couple of different of descision trees

from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import r2_score, mean_squared_error
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_decision_regions


#from imblearn.over_sampling import SMOTE



#from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn import tree


# setting parameters for visualization
plt.rcParams['figure.figsize'] = (16, 5)
plt.style.use('fivethirtyeight')

In [126]:
#loading data
data=pd.read_csv("CleanHealthData.csv")

In [127]:
#getting first 5 rows
data.head()

,pt_age,pt_gender,pt_status,APOE4,DX,CDRSB,ADAS11,ADAS13,MMSE,RAVLT_Immediate,Ventricles,Hippocampus,Whole_Brain,Entorhinal,Mid_Temp
0,81,Male,Married,1.0,AD,4.5,22.00,31.00,20.0,22.0,84599.0,5319.0,1129830.0,1791.0,18422.0
1,81,Male,Married,1.0,AD,6.0,19.00,30.00,24.0,19.0,88580.0,5446.0,1100060.0,2427.0,16972.0
2,81,Male,Married,1.0,AD,3.5,24.00,35.00,17.0,31.0,90099.0,5157.0,1095640.0,1596.0,17330.0
3,81,Male,Married,1.0,AD,8.0,25.67,37.67,19.0,23.0,97420.0,5139.0,1088560.0,1175.0,16398.0
4,75,Male,Married,1.0,AD,6.0,27.33,40.33,20.0,17.0,25704.0,6729.0,875798.0,2050.0,15374.0


In [128]:
# lets check the missing values in the dataset
x = data_result.isnull().sum()
y=(data_result.isnull().sum()/data_result.shape[0])*100
z={'Number of missing values':x,'Percentage of missing values':y}
df=pd.DataFrame(z,columns=['Number of missing values', 'Percentage of missing values'])
df.sort_values(by='Percentage of missing values', ascending=False)

,Number of missing values,Percentage of missing values
pt_age,0,0.0
pt_gender,0,0.0
pt_status,0,0.0
APOE4,0,0.0
DX,0,0.0
CDRSB,0,0.0
ADAS11,0,0.0
ADAS13,0,0.0
MMSE,0,0.0
RAVLT_Immediate,0,0.0


In [129]:
df = data.copy()
target = 'DX'
encode = ['pt_gender','pt_status']

for col in encode:
    dummy = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df,dummy], axis=1)
    del df[col]

target_mapper = {'CN':0,'SMC':1,'EMCI':2,'LMCI':3,'AD':4}
def target_encode(val):
    return target_mapper[val]

df['DX'] = df['DX'].apply(target_encode)

### Identify Depenent and Independent variables

In [130]:
# lets form dependent and independent sets

y = df['DX'] #Dependent variable
x = df.drop(['DX'], axis = 1) #Independent variable
y.head()

0    4
1    4
2    4
3    4
4    4
Name: DX, dtype: int64

### Ensemble after oversampling

In [131]:
Rndf_clf = RandomForestClassifier()
DTree_clf = DecisionTreeClassifier(class_weight="balanced", random_state=1337)
SVC_clf = SVC()
clf = [('SVC_clf',SVC_clf),('DTree_clf',DTree_clf),('Rndf_clf',Rndf_clf)] #list of (str, estimator)

In [132]:
#Splitting the dataset
adb_clf = AdaBoostClassifier(estimator= DTree_clf, n_estimators=1000, learning_rate=0.02, random_state=0)
adb_clf.fit(x, y)

AdaBoostClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                    random_state=1337),
                   learning_rate=0.02, n_estimators=1000, random_state=0)

In [133]:
# Saving the Adaptive Boost model for future prediction
pickle.dump(adb_clf, open('Adb_model_clf.pkl', 'wb'))